Creates Files Making up the Aliased Dataset (Figure 2c, 4, 7, 8)

- Running this notebook will generate thousands of files, and takes ~15 hours at the current resolution.

- These data have already been made for a range of frequencies in ' aliased_dataset/ '

- This code uses the geomoetry of the antenna array and permutations of azimuth and elevation angles to simulate incoming radio signals at different frequencies. 

- These angles are then run through the coherence analysis code ' coherence_analysis.f ' which determines the output azimuth and elevation angles. 

- This result can then be used to simulate regions where angular aliasing can occur using 'figure3_alias_contour_plots.ipynb '

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
elevation_deg = np.arange(5,87,2,dtype=np.float32)
azimuth_deg = np.arange(0,362,2,dtype=np.float32)
elevation = elevation_deg*np.pi/180 
azimuth = azimuth_deg*np.pi/180
ele_string = elevation_deg.astype(int).astype(str)
azi_string = azimuth_deg.astype(int).astype(str)

ele_filenames = []
azi_filenames = []

for i in ele_string:  
    ele_filenames.append(str(i).zfill(4))
    
for i in azi_string:
    azi_filenames.append('0' + str(i).zfill(4))
    
f = [1.0*10**6, 2.8*10**6, 3.9*10**6, 4.1*10**6, 4.3*10**6, 5.25*10**6, 5.4*10**6, 5.8*10**6]
omega = 2*np.pi*f[6]
c = 3.0*10.0**8.0
k_mag = omega/c

41 181 7421


In [3]:
'Creating Reciever Geometry: specific to Toolik Lake Field Station 3-antenna array:'

def format_array(kx_array,ky_array,kz_array):
    
    t = np.linspace(0.,0.03,300000)
    antennas = [0,1,2]
    x_array = np.array([0,44.13,23.34])
    y_array = np.array([0,17.90,66.49])
    dx0= x_array[0]
    dy0= y_array[0]
    dz0= 0.
    dx1= x_array[1]
    dy1= y_array[1]
    dz1= 0.
    dx2= x_array[2]
    dy2= y_array[2]
    dz2= 0.
    dot0_array = kx_array*dx0 + ky_array*dy0 + kz_array*dz0
    dot1_array = kx_array*dx1 + ky_array*dy1 + kz_array*dz1
    dot2_array = kx_array*dx2 + ky_array*dy2 + kz_array*dz2
    y0_array = np.cos(omega*t + dot0_array)
    y1_array = np.cos(omega*t + dot1_array)
    yb_array = np.zeros(np.shape(y0_array))     
    y2_array = np.cos(omega*t + dot2_array)
    total_array = np.array([[y0_array],[y1_array],[yb_array],[y2_array]])
    
    return total_array

In [5]:
def writing_files(file,azi,ele,azi_string,ele_string):
    
    with open(file,"wb") as binary_file:
        binary_file.write(bytes_array)
    with open("/Users/tedipro/Documents/GitHub/Churchill/filename.txt","a") as text_file:
        text_file.write('20230325-0819-0832-TLK-INT.dat \n')
        text_file.write('20230325-0819-0832-TLK-INT.dat 0 2 4598800 4601200') # 4.6 MHz
        
        #Other Frequencies: '
            #text_file.write('20220802-0128-0129-TLK-INT.dat 0 2 1000150 1000450') # 1.0 MHz
            #text_file.write('20220802-0128-0129-TLK-INT.dat 0 2 2799850 2800250') # 2.8 MHz
            #text_file.write('20230325-0819-0832-TLK-INT.dat 0 2 3898800 3901200') # 3.9 MHz
            #text_file.write('20230325-0819-0832-TLK-INT.dat 0 2 4098800 4101200') # 4.1 MHz
            #text_file.write('20230325-0819-0832-TLK-INT.dat 0 2 4198800 4201200') # 4.2 MHz / 5.8 MHz
            #text_file.write('20220802-0128-0129-TLK-INT.dat 0 2 4300050 4300350') # 4.3 MHz
            #text_file.write('20220802-0128-0129-TLK-INT.dat 0 2 4748800 4751200') # 4.75 MHz / 5.25 MHz
    
    #Bash for Coherent Analysis Code:'
    os.system("mv filename.txt filename")
    os.system("./a.out")
    os.system("rm *azi.gnuplot")
    os.system("rm *ele.gnuplot")
    os.system("rm filename")
    remove = "rm {}".format(file)
    os.system(remove)
    rename = "mv *pow.gnuplot {}-{}.txt".format(azi,ele)
    os.system(rename)
    az = azi_string[j].rjust(4)
    el = ele_string[i].rjust(2)
    
    #Creating file for every unique azimuth and elevation combination
    #with open("/Users/tedipro/Documents/GitHub/Churchill/{}-{}.txt".format(azi,ele),"a") as text_file:
        #text_file.write(" {} {}".format(az,el))

NOTE:

Before calling the above functions in the cell bellow: 
    1. Have 'coherence_analysis.f' in the same working directory as this python code.
    2. Navigate a new terminal window to this working directory and compile the Fortran code using:
         ' gfortran --std=legacy coherence_analysis.f '

In [6]:
' Creating the waves, running them through Coherent Analysis, and writing files for each angle combination.'

' Runtimes:  coarse -  18 mins -   362 angles  '
'              fine -    5 hrs -  5695 angles  '
'             1 deg -   15 hrs - 17000 angles  ' #current resolution

for i,ele_i in enumerate(elevation):
    
    kx_ele = k_mag*np.cos(ele_i)
    ky_ele = k_mag*np.cos(ele_i)
    kz = k_mag*np.sin(ele_i)
    ele = ele_filenames[i]
    
    for j,azi_j in enumerate(azimuth):
        
        kx = kx_ele*np.sin(azi_j)
        ky = ky_ele*np.cos(azi_j)
        azi = azi_filenames[j]
        
        total_array = format_array(kx,ky,kz)
        full_length = 16000*np.ndarray.flatten(total_array,order='F')
        test1 = np.array([full_length], dtype='<u2')
        bytes_array = test1.tobytes('F') #Fortran contingent
        file = "/Users/tedipro/Documents/GitHub/Churchill/20230325-0819-0832-TLK-INT.dat"
        writing_files(file,azi,ele,azi_string,ele_string) 